# Hidden Markov Models

https://builtin.com/articles/hidden-markov-model

In [66]:
import numpy as np
import pandas as pd

## 1. Forward algorithm - evaluation problem

### Naive way: go over every single possible combinations of hidden states

See excel for visualised tree diagram

In [49]:
# Initial state probabilities
initial_prob = np.array([0.6, 0.4])  # TIRED, HAPPY

# State transition matrix
transition_prob = np.array([
    [0.7, 0.3],  # TIRED -> [TIRED, HAPPY]
    [0.4, 0.6]   # HAPPY -> [TIRED, HAPPY]
])
states = [0, 1]  # indices of [TIRED, HAPPY]

# Emission probabilities
emission_prob = np.array([
    [0.3, 0.5, 0.2],  # TIRED -> [FAIL, OK, PERFECT]
    [0.1, 0.5, 0.4]   # HAPPY -> [FAIL, OK, PERFECT]
])

# Observation sequence: OK -> FAIL -> PERFECT
observations = [1, 0, 2]

In [ ]:
# t1 =================
# start with first state being TIRED (out of 2 states: TIRED 0.6 AND HAPPY 0.4)
# prob of getting an OK is 0.5, i.e. emission_prob[0, 1] # 1 stands for ok
# t2 =================
# second state = transition_prob[0] (first row, either TIRED to TIRED or TIRED to HAPPY)
# let's start with the first state in the first row: tired to TIRED
# prob of getting a FAIL is 0.3, i.e. emission_prob[0, 0] # 0 stands for fail
# t3 =================
# third state = transition_prob[0] (first row, either TIRED to TIRED or TIRED to HAPPY)
# let's start with the first state in the first row: tired to TIRED
# prob of getting a PERFECT is 0.2, i.e. emission_prob[0, 2] # 2 stands for perfect

# total prob of scenario ==================
# = 0.5 * 0.3 * 0.2 = 0.03

# back to t3 =================
# let's move on to the second state in the first row: tired to HAPPY
# back to t3 =================
# let's move on to the first state in the second row: happy to TIRED
# back to t3 =================
# let's move on to the second state in the second row: happy to HAPPY

# back to t2 =================
# let's start with the second state in the first row: tired to HAPPY
# back to t2 =================
# let's start with the first state in the second row: happy to TIRED
# back to t2 =================
# let's start with the second state in the second row: happy to HAPPY

# back to t1 =================
# let's start with the second state in the first row: tired to HAPPY
# back to t1 =================
# let's start with the first state in the second row: happy to TIRED
# back to t1 =================
# let's start with the second state in the second row: happy to HAPPY

In [ ]:
for s1 in states: 
    for s2 in states:
        for s3 in states:
            state_prob = transition_prob[s2, s3]
            observation_prob = emission_prob[s3, observations[2]]

### Forward algorithm

https://towardsdatascience.com/hidden-markov-models-explained-with-a-real-life-example-and-python-code-2df2a7956d65

In [81]:
# Initial state probabilities
initial_prob = np.array([0.6, 0.4])  # TIRED, HAPPY

# State transition matrix
transition_prob = np.array([
    [0.7, 0.3],  # TIRED -> [TIRED, HAPPY]
    [0.4, 0.6]   # HAPPY -> [TIRED, HAPPY]
])

# Emission probabilities
emission_prob = np.array([
    [0.3, 0.5, 0.2],  # TIRED -> [FAIL, OK, PERFECT]
    [0.1, 0.5, 0.4]   # HAPPY -> [FAIL, OK, PERFECT]
])

# Observation sequence: OK -> FAIL -> PERFECT
observations = [1, 0, 2]

# Forward algorithm
# Initialize alpha values
alpha = np.zeros((len(observations), len(initial_prob)))

# Initial step
for s in range(len(initial_prob)):
    alpha[0, s] = initial_prob[s] * emission_prob[s, observations[0]]

# Recursion step
for t in range(1, len(observations)):
    for s in range(len(initial_prob)):
        alpha[t, s] = np.sum(alpha[t - 1] * transition_prob[:, s]) * emission_prob[s, observations[t]]

# Termination step: sum of the final alpha values
probability = np.sum(alpha[-1])

print(f"The probability of observing the sequence OK -> FAIL -> PERFECT is: {probability}")

The probability of observing the sequence OK -> FAIL -> PERFECT is: 0.02934


In [82]:
from hmmlearn import hmm
import numpy as np

## Part 1. Generating a HMM with specific parameters and simulating the exam
print("Setup HMM model with parameters")
# init_params are the parameters used to initialize the model for training
# s -> start probability
# t -> transition probabilities
# e -> emission probabilities
model = hmm.CategoricalHMM(n_components=2, random_state=425, init_params='ste')

# initial probabilities
# probability of starting in the Tired state = 0
# probability of starting in the Happy state = 1
initial_distribution = np.array([0.1, 0.9])
model.startprob_ = initial_distribution

print("Step 1. Complete - Defined Initial Distribution")

# transition probabilities
#        tired    happy
# tired   0.4      0.6
# happy   0.2      0.8

transition_distribution = np.array([[0.4, 0.6], [0.2, 0.8]])
model.transmat_ = transition_distribution
print("Step 2. Complete - Defined Transition Matrix")

# observation probabilities
#        Fail    OK      Perfect
# tired   0.3    0.5       0.2
# happy   0.1    0.5       0.4

observation_probability_matrix = np.array([[0.3, 0.5, 0.2], [0.1, 0.5, 0.4]])
model.emissionprob_ = observation_probability_matrix
print("Step 3. Complete - Defined Observation Probability Matrix")

# simulate performing 100,000 trials, i.e., aptitude tests
trials, simulated_states = model.sample(100000)

# Output a sample of the simulated trials
# 0 -> Fail
# 1 -> OK
# 2 -> Perfect
print("\nSample of Simulated Trials - Based on Model Parameters")
print(trials[:10])

## Part 2 - Decoding the hidden state sequence that leads
## to an observation sequence of OK - Fail - Perfect

# split our data into training and test sets (50/50 split)
X_train = trials[:trials.shape[0] // 2]
X_test = trials[trials.shape[0] // 2:]

model.fit(X_train)

# the exam had 3 trials and your dog had the following score: OK, Fail, Perfect (1, 0 , 2)
exam_observations = [[1, 0, 2]]
predicted_states = model.predict(X=[[1, 0, 2]])
print("Predict the Hidden State Transitions that were being the exam scores OK, Fail, Perfect: \n 0 -> Tired , "
      "1 -> Happy")
print(predicted_states)

Setup HMM model with parameters
Step 1. Complete - Defined Initial Distribution
Step 2. Complete - Defined Transition Matrix
Step 3. Complete - Defined Observation Probability Matrix


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'emissionprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'e'



Sample of Simulated Trials - Based on Model Parameters
[[1]
 [1]
 [2]
 [1]
 [2]
 [0]
 [1]
 [1]
 [1]
 [2]]
Predict the Hidden State Transitions that were being the exam scores OK, Fail, Perfect: 
 0 -> Tired , 1 -> Happy
[1 0 0]


In [1]:
import numpy as np
obs_to_idx = {'normal':0, 'cold': 1, 'dizzy':2}
state_to_idx = {'Healthy':0, 'Fever':1}

In [2]:
pi = np.array([0.6, 0.4])

# transition probabilities
A = np.array([[0.7, 0.3],
              [0.4, 0.6]])

# emission probabilties
B = np.array([[0.5, 0.4, 0.1],
              [0.1, 0.3, 0.6]])

X = [['normal', 'cold', 'dizzy'], 
     ['normal', 'cold', 'cold'],
     ['cold', 'cold', 'dizzy'],
     ['normal', 'normal', 'normal']]